# Sequence Model and LSTMs

Read it here 
https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

Till now what we have seen is that there no state maintained by
the networks at all

sequence models are very critical 
here we want to store some sort of dependence 
through time between your inputs 

rnn model keeps some state h_t for each element in the sequence 
h_t keeps some information about the previous data 


## Pytorch and LSTMs
All pytorch inputs to LSTM's should be 3D tensors 

>- first axis is sequence  
>- second axis is mini-batch
>- third axis is index of input 



In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

torch.manual_seed(1)

In [17]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

count = 0
#input shape: seq_len x 1 x inputs
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print("seq : {}\n  output layer: {}".format(count,out.data))
    print("hidden layer 0: {} \nhidden layer 1: {}".format(hidden[0].data,hidden[1].data))
    count +=1

seq : 0
  output layer: tensor([[[ 0.7334,  0.3474, -0.0863]]])
hidden layer 0: tensor([[[ 0.7334,  0.3474, -0.0863]]]) 
hidden layer 1: tensor([[[ 1.7581,  0.4428, -0.1023]]])
seq : 1
  output layer: tensor([[[ 0.4564, -0.1400, -0.2439]]])
hidden layer 0: tensor([[[ 0.4564, -0.1400, -0.2439]]]) 
hidden layer 1: tensor([[[ 0.9739, -0.1904, -0.3135]]])
seq : 2
  output layer: tensor([[[ 0.3572, -0.2635, -0.2943]]])
hidden layer 0: tensor([[[ 0.3572, -0.2635, -0.2943]]]) 
hidden layer 1: tensor([[[ 0.5302, -0.4634, -0.4558]]])
seq : 3
  output layer: tensor([[[ 0.1545,  0.0893, -0.0656]]])
hidden layer 0: tensor([[[ 0.1545,  0.0893, -0.0656]]]) 
hidden layer 1: tensor([[[ 0.2552,  0.1104, -0.0917]]])
seq : 4
  output layer: tensor([[[ 0.1946, -0.1276, -0.3476]]])
hidden layer 0: tensor([[[ 0.1946, -0.1276, -0.3476]]]) 
hidden layer 1: tensor([[[ 0.2526, -0.4293, -0.5375]]])


In [24]:
print(len(inputs))
inputs

5


[tensor([[ 0.7890,  1.1363, -0.1147]]),
 tensor([[-0.0180,  0.0491,  0.4272]]),
 tensor([[-0.8996,  0.5313,  0.4034]]),
 tensor([[ 1.4521, -2.4182, -1.1906]]),
 tensor([[0.6964, 1.1296, 0.2214]])]

In [25]:
hidden

(tensor([[[-0.0145, -0.0034,  0.3334]]], grad_fn=<ViewBackward>),
 tensor([[[-0.0356, -0.0072,  0.6871]]], grad_fn=<ViewBackward>))

In [26]:
out

tensor([[[-0.0145, -0.0034,  0.3334]]], grad_fn=<CatBackward>)

In [6]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


In [15]:
print(out)
print("Last layer : \n{}".format(out[-1]))
print("Hidden layer  \n{}".format(hidden))

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
Last layer : 
tensor([[-0.3368,  0.0959, -0.0538]], grad_fn=<SelectBackward>)
Hidden layer  
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


In [36]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
print(training_data)
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]
{'dog': 1, 'ate': 2, 'read': 6, 'the': 3, 'apple': 4, 'that': 7, 'book': 8, 'The': 0, 'Everybody': 5}


In [60]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print("Embedding dim: {}".format(embeds.shape))
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        print("LSTM out shape:{}".format(lstm_out.shape))
        print("hidden :{}".format(hidden))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        print("tag_space : {}".format(tag_space))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [61]:
model = model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
model

LSTMTagger(
  (word_embeddings): Embedding(9, 6)
  (lstm): LSTM(6, 6)
  (hidden2tag): Linear(in_features=6, out_features=3, bias=True)
)

In [63]:
#model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print("TAG SCORES before training: \n{}".format(tag_scores))

for epoch in range(3):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

Embedding dim: torch.Size([5, 6])
LSTM out shape:torch.Size([5, 1, 6])
hidden :(tensor([[[-0.0145, -0.0034,  0.3334]]], grad_fn=<ViewBackward>), tensor([[[-0.0356, -0.0072,  0.6871]]], grad_fn=<ViewBackward>))
tag_space : tensor([[-0.1854,  0.1023, -0.2464],
        [-0.1842,  0.0884, -0.1563],
        [-0.2289,  0.1244, -0.1270],
        [-0.1396,  0.0581, -0.2744],
        [-0.1028,  0.0537, -0.3059]])
TAG SCORES before training: 
tensor([[-1.1861, -0.8983, -1.2471],
        [-1.2065, -0.9338, -1.1786],
        [-1.2616, -0.9083, -1.1597],
        [-1.1289, -0.9313, -1.2638],
        [-1.0938, -0.9373, -1.2969]])
Embedding dim: torch.Size([5, 6])
LSTM out shape:torch.Size([5, 1, 6])
hidden :(tensor([[[-0.0145, -0.0034,  0.3334]]], grad_fn=<ViewBackward>), tensor([[[-0.0356, -0.0072,  0.6871]]], grad_fn=<ViewBackward>))
tag_space : tensor([[-0.2155,  0.1076, -0.2071],
        [-0.1968,  0.0869, -0.1487],
        [-0.2385,  0.1227, -0.1248],
        [-0.1417,  0.0526, -0.2739],
       